In [ ]:
import json
import os
import numpy as np
from docarray import BaseDoc, DocList  # corrigindo nomes modernos
from typing import List
from numpy.typing import NDArray
from collections import defaultdict
from sentence_transformers import SentenceTransformer
import faiss
from fastapi import FastAPI
from pydantic import BaseModel
import pickle, os



In [2]:
K_VIZINHOS = 5  

DIRETORIO_SAIDA = "indices_chatbot"
NOME_ARQUIVO_INDICE = "faiss_index_{cultura}.index"
NOME_ARQUIVO_METADADOS = "metadata_{cultura}.json"

# Garante que o diretório para salvar os arquivos exista
os.makedirs(DIRETORIO_SAIDA, exist_ok=True)

In [3]:
bulks = {
    "abacaxi": "bulk/bulk-abacaxi.json",
    "uva": "bulk/bulk-uva.json",
    "milho": "bulk/bulk-milho.json"
}



In [4]:
from  glob import glob
glob('data/book/*')

[]

In [5]:
import os
os.getcwd()


'/home/sintik/sadai-chat'

In [6]:
class QA_Doc(BaseDoc):
    text: str
    index: str | None = None
    question_number: int | None = None
    question: str | None = None
    answer: str | None = None
    chapter: str | None = None
    book: str | None = None
    book_id: str | None = None
    epub: str | None = None
    pdf: str | None = None
    html: str | None = None
    year: str | None = None
    embedding: list[float] | None = None
    

def carregar_doc(jsonl_path, cultura=None):
    docs = []

    with open(jsonl_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # lê as linhas de 2 em 2 (index + data)
    for i in range(0, len(lines), 2):
        index_line = json.loads(lines[i])
        data_line = json.loads(lines[i + 1])

        doc = QA_Doc(
            text=f"{data_line.get('question', '')}\n{data_line.get('answer', '')}",
            index=index_line["index"]["_id"],
            question_number=data_line.get("question_number"),
            question=data_line.get("question"),
            answer=data_line.get("answer"),
            chapter=data_line.get("chapter"),
            book=data_line.get("book"),
            book_id=data_line.get("book_id"),
            epub=data_line.get("epub"),
            pdf=data_line.get("pdf"),
            year=str(data_line.get("year")) if data_line.get("year") is not None else "",
        )

        # não usamos 'doc.cultura' mais
        # doc.cultura = cultura
        # doc.question = data_line.get("question", "")  # já está setado no construtor

        docs.append(doc)

    return DocList[QA_Doc](docs)

In [7]:

doc_cultura = defaultdict(list)

for cultura, caminho in bulks.items():
    print(f" Carregando documentos de {cultura}...")
    docs = carregar_doc(caminho, cultura)
    doc_cultura[cultura].extend(docs)

# Mostra resumo de quantos docs por cultura
for cultura, docs in doc_cultura.items():
    print(f"Cultura: {cultura} -> {len(docs)} documentos")
    if len(docs) > 0:
        print("Exemplo:", docs[0].question)
    print()


 Carregando documentos de abacaxi...
 Carregando documentos de uva...
 Carregando documentos de milho...
Cultura: abacaxi -> 500 documentos
Exemplo: A falta de chuva prejudica o abacaxizeiro?

Cultura: uva -> 500 documentos
Exemplo: Quais são os métodos usados no melhoramento genético da videira?

Cultura: milho -> 500 documentos
Exemplo: Como o clima influencia a cultura do milho?



In [ ]:
# Garante diretório de saída
os.makedirs("./dados", exist_ok=True)

# Caminho do arquivo
CAMINHO_PKL = "./dados/doc_cultura.pkl"

# Salva a estrutura de documentos (persistência)
with open(CAMINHO_PKL, "wb") as f:
    pickle.dump(doc_cultura, f)

print(f"Estrutura 'doc_cultura' salva em {CAMINHO_PKL}")


Estrutura 'doc_cultura' salva em ./dados/doc_cultura.pkl


In [9]:
if not isinstance(doc_cultura, dict) or len(doc_cultura) == 0:
    raise ValueError(" doc_cultura está vazio ou inválido!")
else:
    print(f"doc_cultura carregado com {len(doc_cultura)} culturas.")


doc_cultura carregado com 3 culturas.


In [ ]:

# Criação e persistência dos índices FAISS usando SentenceTransformer


model = SentenceTransformer(
    "PORTULAN/serafim-335m-portuguese-pt-sentence-encoder-ir"
)

def gerar_embeddings(textos):
    """
    Gera embeddings normalizados usando SentenceTransformer.
    Já retorna float32 normalizado (N-esfera).
    """
    emb = model.encode(
        textos,
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    return emb.astype("float32")

# === Diretório onde os índices serão salvos ===
DIRETORIO_SAIDA = "./indices_faiss"
os.makedirs(DIRETORIO_SAIDA, exist_ok=True)

NOME_ARQUIVO_INDICE = "index_{cultura}.faiss"
NOME_ARQUIVO_METADADOS = "metadata_{cultura}.json"

# Índices FAISS por cultura
faiss_index_cultura = {}
metadados_cultura = {}

for cultura, docs in doc_cultura.items():

    print(f"\n=== Cultura: {cultura} ===")

    caminho_indice = os.path.join(DIRETORIO_SAIDA, NOME_ARQUIVO_INDICE.format(cultura=cultura))
    caminho_meta   = os.path.join(DIRETORIO_SAIDA, NOME_ARQUIVO_METADADOS.format(cultura=cultura))

    # Se já existir, só carrega
    if os.path.exists(caminho_indice) and os.path.exists(caminho_meta):
        print("Carregando índice existente...")

        index = faiss.read_index(caminho_indice)
        faiss_index_cultura[cultura] = index

        with open(caminho_meta, "r", encoding="utf-8") as f:
            metadados_cultura[cultura] = json.load(f)

        continue

    # Caso contrário, cria do zero
    print("Criando novo índice...")

    textos = [f"{doc.question} {doc.answer}" for doc in docs]
    embeddings = gerar_embeddings(textos)

    dimensao = embeddings.shape[1]
    index_flat = faiss.IndexFlatL2(dimensao)
    index = faiss.IndexIDMap(index_flat)

    ids = np.arange(len(docs)).astype("int64")
    index.add_with_ids(embeddings, ids)
    faiss_index_cultura[cultura] = index

    # Salva índice FAISS
    faiss.write_index(index, caminho_indice)

    # Salva metadados
    metadados = {
        str(i): {
            "question": doc.question,
            "answer": doc.answer,
            "book": doc.book,
        }
        for i, doc in enumerate(docs)
    }

    with open(caminho_meta, "w", encoding="utf-8") as f:
        json.dump(metadados, f, ensure_ascii=False, indent=2)

    metadados_cultura[cultura] = metadados

    print("Índice e metadados salvos.")

print("\n✓ Todos os índices FAISS foram carregados ou criados com sucesso.")



=== Cultura: abacaxi ===
Criando novo índice...
Índice e metadados salvos.

=== Cultura: uva ===
Criando novo índice...
Índice e metadados salvos.

=== Cultura: milho ===
Criando novo índice...
Índice e metadados salvos.

✓ Todos os índices FAISS foram carregados ou criados com sucesso.


In [ ]:
#Função para detectar a cultura mais próxima


def detectar_cultura(query, embeddings_por_cultura):
    # 1. Calcula a média dos embeddings de cada cultura
    medias_cultura = {
        cultura: np.mean(np.array(embs), axis=0)
        for cultura, embs in embeddings_por_cultura.items()
    }

    # 2. Gera o embedding da query usando o modelo Serafim
    query_emb = gerar_embeddings([query])[0] 

    # 3. Calcula similaridade coseno entre query e médias
    similaridades = {}
    for cultura, media in medias_cultura.items():
        media_norm = media / np.linalg.norm(media)
        sim = np.dot(query_emb, media_norm)
        similaridades[cultura] = sim

    # 4. Determina cultura mais próxima
    cultura_mais_proxima = max(similaridades, key=similaridades.get)

    
    print(" Similaridades por cultura:")
    for c, s in similaridades.items():
        print(f"   {c}: {s:.4f}")
    print(f" Cultura mais próxima: {cultura_mais_proxima}")

    return cultura_mais_proxima


In [13]:
#  Buscar documentos apenas na cultura mais próxima

def buscar_documentos_por_cultura(query, embeddings_por_cultura, k=5):
    # Detecta cultura mais próxima
    cultura_escolhida = detectar_cultura(query, embeddings_por_cultura)

    # Recupera índice e documentos da cultura
    index = faiss_index_cultura[cultura_escolhida]
    docs_cultura = doc_cultura[cultura_escolhida]

    # Gera embedding da query
    query_emb = gerar_embeddings([query]).astype('float32')
    faiss.normalize_L2(query_emb)

    # Busca no índice FAISS
    D, I = index.search(query_emb, k)
    print(f"\n Resultados da busca na cultura: {cultura_escolhida}\n")

    # Monta resultados legíveis
    resultados = []
    for idx, dist in zip(I[0], D[0]):
        if idx < len(docs_cultura):
            pergunta = docs_cultura[idx].tags.get("question", "")
            resposta = docs_cultura[idx].tags.get("answer", "")
            resultados.append({
                "pergunta": pergunta,
                "resposta": resposta,
                "distancia": float(dist)
            })
            print(f"🔹 {pergunta} (distância={dist:.4f})")

    return resultados


In [ ]:
# Função de Busca Semântica entre todas as culturas


def buscar_documentos_relevantes(query, k=5):
    # Gera embedding da query
    query_emb = gerar_embeddings([query]).astype('float32')
    resultados = []

    #  Busca em cada índice FAISS por cultura
    for cultura, index in faiss_index_cultura.items():
        D, I = index.search(query_emb, k)

        for idx, distancia in zip(I[0], D[0]):
            if idx < len(doc_cultura[cultura]):  # Verifica índice válido
                doc = doc_cultura[cultura][idx]
                resultados.append({
                    'doc': doc,
                    'cultura': cultura,
                    'distancia': float(distancia)
                })

    # Ordena os resultados por distância (quanto menor, mais próximo)
    resultados.sort(key=lambda x: x['distancia'])

    # Seleciona os top-k documentos mais próximos
    top_docs = [r['doc'] for r in resultados[:k]]

    #  Determina cultura predominante entre os top-k
    if top_docs:
        culturas_resultados = [r['cultura'] for r in resultados[:k]]
        cultura_predominante = max(set(culturas_resultados), key=culturas_resultados.count)
    else:
        cultura_predominante = "abacaxi"  

    print(f" Cultura predominante: {cultura_predominante}")
    return top_docs, cultura_predominante

print(" Função de busca semântica implementada com sucesso!")


 Função de busca semântica implementada com sucesso!


In [15]:
# Dicionário para guardar embeddings médios de cada cultura para detecção de cultura
embeddings_por_cultura = {}

for cultura, docs in doc_cultura.items():
    if len(docs) == 0:
        print(f" Nenhum documento encontrado para {cultura}, pulando...")
        continue

    print(f" Gerando embeddings para {cultura}...")
    textos = [f"{doc.question} {doc.answer}" for doc in docs]
    embeddings = gerar_embeddings(textos)  # usa a MESMA função que o FAISS usa
    embeddings_por_cultura[cultura] = embeddings
    print(f" {len(embeddings)} embeddings criados para {cultura}\n")

print(" Embeddings coletados para todas as culturas!")


# Calcula a média por cultura (para detectar cultura mais próxima)
medias_cultura = {
    cultura: np.mean(embeds, axis=0)
    for cultura, embeds in embeddings_por_cultura.items()
}
print("Médias de embeddings calculadas!")


 Gerando embeddings para abacaxi...
 500 embeddings criados para abacaxi

 Gerando embeddings para uva...
 500 embeddings criados para uva

 Gerando embeddings para milho...
 500 embeddings criados para milho

 Embeddings coletados para todas as culturas!
Médias de embeddings calculadas!
